## AoC 2023 - Day 20

### Setup imports and get raw data

In [129]:
from collections import deque
from copy import deepcopy

from icecream import ic
from pprint import pprint

ic.enable()

data = open('input.txt').read().split('\n')

### Parse the input to generate modules dict

In [130]:
modules = {}
broadcast_dest = []
for m in data:
    name, _, dest = m.split(maxsplit=2)
    myname = name[1:]
    if name == "broadcaster":
        broadcast_dest = dest.replace(' ', '').split(',')
        myname = '_b'
        this = {'type': 'b', 'dest': broadcast_dest}
    elif name[0] == "%":
        this = {'type': name[0], 'dest': dest.replace(' ', '').split(','), 'state': 0}
    elif name[0] == "&":
        this = {'type': name[0], 'dest': dest.replace(' ', '').split(','), 'states': {}}

    modules[myname] = this

for m in [x for x in modules if modules[x]["type"] == '&']:
    #ic(modules[m])
    modules[m]["states"] = {y: 0 for y in modules if m in modules[y]["dest"]}

modules['output'] = {'type': "o", 'state': 0}
null = ic(broadcast_dest)
null = ic(modules)
#null = ic(len(modules['inv']["states"]))

ic| broadcast_dest: ['a']
ic| modules: {'_b': {'dest': ['a'], 'type': 'b'},
              'a': {'dest': ['inv', 'con'], 'state': 0, 'type': '%'},
              'b': {'dest': ['con'], 'state': 0, 'type': '%'},
              'con': {'dest': ['output'], 'states': {'a': 0, 'b': 0}, 'type': '&'},
              'inv': {'dest': ['b'], 'states': {'a': 0}, 'type': '&'},
              'output': {'state': 0, 'type': 'o'}}


### Pseudo code of process

1. Setup Pulse queue  
2. Push the initial broadcast pulses to the queue  
3. While there's items in the queue  
  1. Popleft an item off the queue  
  2. If it's a flip-flop %  
     if the pulse is low, flip state  
      - if it was off, queue a high pulse  
      - if it was on, queue a low pulse  
      - increment counters for sent pulses  
  3. If it's a conjunction &  
    1. store the state of the inbound pulse  
    2. if all pulses are High, queue a low pulse  
    3. else queue a high pulse  
    4. increment counters for sent pulses  

In [131]:


pulses_high = 0
pulses_low = 0
modules_cycle = deepcopy(modules)
for _ in range(1000):
    #print('\n-- START CYCLE --')
    #modules_cycle = deepcopy(modules)

    #print(f'___button -low-> broadcaster')
    queue = deque([('broadcast', d, 0) for d in broadcast_dest])

    pulses_low += 1 + len(broadcast_dest)

    #null = ic(queue)


    ic.disable()
    while queue:
        #ic('Start')
        #cycle_modules = deepcopy(modules_cycle)
        s, d, t = ic(queue.popleft())

        #print(f'___{s} {"-low->" if t == 0 else "-high->"} {d}')
        # Flip-flop, but only if the pulse is low
        if modules_cycle[d]['type'] == '%' and t == 0:
            # flip our state high -> low, low -> high
            modules_cycle[d]['state'] = 0 if modules_cycle[d]['state'] == 1 else 1
            #queue up a pulse to each of our destinations
            # pulse type is the same as our (new) state
            new = [(d, x, modules_cycle[d]['state']) for _, x in enumerate(modules_cycle[d]['dest'])]
            

            #queue.extend(new)
            if modules_cycle[d]['state'] == 1:
                pulses_high += len(new)
            else:
                pulses_low += len(new)
            ic(' % adding', new)
            queue.extend(new)
            
        # conjunction
        elif modules_cycle[d]['type'] == '&':
            modules_cycle[d]['states'][s] = t
            if sum([foo for foo in modules_cycle[d]['states'].values()]) == len(modules_cycle[d]["states"]):
                # All input states are high, queue a low pulse to all our destinations
                new = [(d, x, 0) for x in modules_cycle[d]['dest']]
                pulses_low += len(new)
            else:
                new = [(d, x, 1) for x in modules_cycle[d]['dest']]
                pulses_high += len(new)
            ic(' & adding', new)
            queue.extend(new)
        
        
        #queue.extend(new)
        
        #null = ic(queue)
        #modules_cycle = cycle_modules
pulses_low, pulses_high

pulses_low * pulses_high


11687500

In [132]:
foobar = deque([1, 2, 3])
#ic.configureOutput(includeContext=False)
null = ic(foobar)
ic(foobar.popleft())

foobar.extend([4, 5, 6])
null = ic(foobar)
ic(foobar.popleft())
ic(foobar.popleft())
ic(foobar.popleft())
ic(foobar.popleft())

5